## Define a new Function
OpenAI chat completion may pick between generating a reply message, or a function call.
This provides an assistant with the ability to define a new function that its model may call.

In [3]:
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
from autogen.code_utils import execute_code

config_list = config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        # Function calling with GPT 3.5
        "model": ["gpt-3.5-turbo-16k-0613"],
    },
)
llm_config = {
    "functions": [
        {
            "name": "define_function",
            "description": "Define a function to add to the context of the conversation. Once defined, the assistant may decide to use this function, respond with a normal message.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "The name of the function to define."
                    },
                    "description": {
                        "type": "string",
                        "description": "A short description of the function."
                    },
                    "packages": {
                        "type": "string",
                        "description": "A list of packages used by the function. These need to be installed with pip prior to invoking the function."
                    },
                    "code": {
                        "type": "string",
                        "description": "The implementation body of the Python function."
                    }
                },
                "required": ["name", "code"],
            },
        },
    ],
    "config_list": config_list,
    "request_timeout": 120,
}

def define_function(name, description, packages, code):
    function_config = {
        "name": name,
        "description": description,
        "parameters": {
            "type": "object",
            "properties": {},
            "required": [],
        },
    }
    llm_config["functions"] = llm_config["functions"] + [function_config]
    user_proxy.register_function(
        function_map={
            name: lambda: execute_func(name, packages, code)
        }
    )
    assistant.define_function(function_config)
    return f"A function has been added to the context of this conversation.\nDescription: {description}\nAs an assistant, you should now request further instructions, or reply TERMINATE if the initial request has been fulfilled."

def execute_func(name, packages, code):
    str = f"""
import subprocess
subprocess.run(["pip", "install", "{packages}"])
{code}
print({name}())
"""
    print(f"execute_code:\n{str}")
    result = execute_code(str)[1]
    print(f"Result: {result}")
    return result

def _is_termination_msg(message):
    """Check if a message is a termination message."""
    if isinstance(message, dict):
        message = message.get("content")
        if message is None:
            return False
        return message.rstrip().endswith("TERMINATE")

assistant = AssistantAgent(
    name="chatbot",
    system_message="""You are an assistant.
        The user will ask a question.
        You may use the provided functions before providing a final answer.
        Only use the functions you were provided.
        When the answer has been provided, reply TERMINATE.""",
    llm_config=llm_config,
)

user_proxy = UserProxyAgent(
    "user_proxy",
    code_execution_config={"work_dir": "coding", "use_docker": True},
    is_termination_msg=_is_termination_msg,
    default_auto_reply="Reply TERMINATE when the initial request has been fulfilled.",
    human_input_mode="NEVER")

user_proxy.register_function(
    function_map={
        "define_function": define_function
    }
)

# user_proxy.initiate_chat(
#     assistant, message="What functions do you know about?")

user_proxy.initiate_chat(
    assistant, message="Define a function that prints the response body. Use https://echo.free.beeceptor.com/")

# user_proxy.initiate_chat(
#     assistant, message="List functions do you know about.")

user_proxy.initiate_chat(
    assistant, message="Print the response body. Use the functions you know about.")


user_proxy (to chatbot):

Define a function that prints the response body. Use https://echo.free.beeceptor.com/

--------------------------------------------------------------------------------


[{'description': 'Define a function to add to the context of the conversation. '
                 'Once defined, the assistant may decide to use this function, '
                 'respond with a normal message.',
  'name': 'define_function',
  'parameters': {'properties': {'code': {'description': 'The implementation '
                                                        'body of the Python '
                                                        'function.',
                                         'type': 'string'},
                                'description': {'description': 'A short '
                                                               'description of '
                                                               'the function.',
                                                'type': 'string'},
                                'name': {'description': 'The name of the '
                                                        'function to define.',
                 